In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
import numpy as np
import random
from torch import nn
from glob import glob
from tqdm.auto import tqdm
import pandas as pd
torch.cuda.is_available()

True

In [5]:
from maatool.data.feats_itdataset_v2 import FeatsIterableDatasetV2
from maatool.models.transformer import TransformerWithSinPos

In [6]:
import pytorch_lightning as pl 

In [6]:
import logging
import logging.config

def configure_logging(log_level):
    handlers =  {
            "maa": {
                "class": "logging.StreamHandler",
                "formatter": "maa_basic",
                "stream": "ext://sys.stdout",
            }
    }
    CONFIG = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {"maa_basic": {"format": '%(asctime)s %(name)s %(pathname)s:%(lineno)d - %(levelname)s - %(message)s'}},
        "handlers": handlers,
        "loggers": {"maa": {"handlers": handlers.keys(), "level": log_level}},
        "root": {"handlers": handlers.keys(), "level": log_level}
    }
    logging.config.dictConfig(CONFIG)
configure_logging("INFO")

In [7]:
torch.distributed.is_initialized()

False

In [8]:
from collections import defaultdict

In [9]:
def get_new_tgt(prev_tgt, hyp_logprobs, logits, topk=4):
    """
    prev_tgt - (T, N_hyp)
    hyp_logprobs - (N_hyp, )
    logits - (N_hyp, C)
    """
    assert len(prev_tgt.shape) == 2, f"{prev_tgt.shape=}"
    assert len(hyp_logprobs.shape) == 1, f"{hyp_logprobs.shape=}"
    assert len(logits.shape) == 2, f"{logits.shape=}"
    assert prev_tgt.shape[1] == hyp_logprobs.shape[0] == logits.shape[0], (
        f"{prev_tgt.shape=} {hyp_logprobs.shape=} {logits.shape=}"
    )
        
    nt_topk_logits, nt_topk_idx = logits.topk(k=topk, axis=-1)
    #print("nt_topk_idx", nt_topk_idx, nt_topk_idx.shape)
    # (N, K)
    next_tokens = nt_topk_idx.T.reshape(1, -1)
    #print("next_tokens", next_tokens, next_tokens.shape)
    # (1, N*(repeat k times)) 
    # (T, N*(repeat k times))  
    new_hyp_tgt = torch.concatenate([prev_tgt.repeat(1, topk), next_tokens], axis=0)
    #print(f"{new_hyp_tgt=}", new_hyp_tgt.shape)
    # (T+1, N*(repeat k times))
    new_scores = nt_topk_logits.T.reshape(-1)
    # N*(repeat k times)
    prew_scores = hyp_logprobs.repeat(topk)
    #print("prew_scores", prew_scores)
    # N*(repeat k times)
    new_hyp_logprob = prew_scores + new_scores
    #print("new_hyp_logprob", new_hyp_logprob)
    new_hyp_logprob, idx = new_hyp_logprob.topk(k=topk)
    #print(idx)
    new_hyps = new_hyp_tgt[:, idx]
    #print("new_hyps", new_hyps, new_hyp_logprob)
    # (T+1, N*k), (N,)
    return new_hyps, new_hyp_logprob
    

    
tgt, logits = get_new_tgt(torch.LongTensor([[1,]]), torch.tensor([-1.]), torch.tensor([[-3, -4, -7, -2, -5]]))
print(">>>>\n", tgt, logits)
get_new_tgt(tgt, logits, torch.tensor([[100,    110,  200], 
                                       [100,    110,  200], 
                                       [100,    110,  200], 
                                       [100,    110,  200]]), topk=2)

>>>>
 tensor([[1, 1, 1, 1],
        [3, 0, 1, 4]]) tensor([-3., -4., -5., -6.])


(tensor([[1, 1],
         [3, 0],
         [2, 2]]),
 tensor([197., 196.]))

In [10]:
def sep_ready_tgt(tgt, logprobs, eos_id=2):
    """
    tgt - (T, N)
    logprobs - (N,)
    """
    assert tgt.shape[1] == logprobs.shape[0], (
        f"{tgt.shape=} {logprobs.shape=}"
    )
    
    is_end_mask = ((tgt == eos_id).sum(axis=0) > 0)
    # (N,)
    #print(is_end_mask)
    ready_tgt = tgt[:, is_end_mask]
    ready_logprobs = logprobs[is_end_mask]
    
    ready_list = [(l.cpu().item(), t.cpu().tolist()) for l, t in zip(ready_logprobs, ready_tgt.T)]

    not_ready_tgt = tgt[:, ~is_end_mask]
    not_ready_logprobs = logprobs[~is_end_mask]
    assert not_ready_tgt.shape[1] == not_ready_logprobs.shape[0], (
        f"{not_ready_tgt.shape[1]=} {not_ready_logprobs.shape[0]=}"
    )
    return ready_list, not_ready_tgt, not_ready_logprobs


sep_ready_tgt(torch.LongTensor([[1, 1], [2, 3]]), torch.tensor([-1., -3]))

([(-1.0, [1, 2])],
 tensor([[1],
         [3]]),
 tensor([-3.]))

In [11]:
def set_random_seed(seed):
    if seed < 0:
        seed = seed_from_time()
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
set_random_seed(42)

class SwipeTransformerRecognizer(pl.LightningModule):
    def __init__(self, backbone, learning_rate=1e-4, speed=42):
        super().__init__()
        self.save_hyperparameters(ignore=['backbone'])
        self.backbone = backbone
        self.ce_loss = nn.CrossEntropyLoss(ignore_index=0, reduction='mean')
        set_random_seed(speed)

    def forward(self, **kwargs):
        return self.backbone(**kwargs)
    
    def get_loss(self, batch):
        # batch - (Time, Batch, ...)
        feats = batch['feats']
        # (Time, Batch, num_feats)
        tgt = batch['targets'][:-1]
        tgt_key_padding_mask = batch['tgt_key_padding_mask'][:, 1:] 
        # (Batch, Seq-1)
        logits = self.backbone(feats=feats, 
                               tgt=tgt, 
                               src_key_padding_mask=batch['src_key_padding_mask'], 
                               tgt_key_padding_mask=tgt_key_padding_mask) 
        # (Seq-1, Batch, C)
        S, N, C = logits.shape
        targets = batch['targets'][1:]
        # (Seq-1, Batch)
        # print("loss ", logits.shape, targets.shape)
        loss = self.ce_loss(logits.view(-1, C), targets.reshape(-1))
        
        return loss

        
    def training_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True,  batch_size=len(batch['uids']))
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('valid_loss', loss, on_step=True,  batch_size=len(batch['uids']))

    def test_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('test_loss', loss,  batch_size=len(batch['uids']))

    def configure_optimizers(self):
        # self.hparams available because we called self.save_hyperparameters()
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--learning_rate', type=float, default=0.0001)
        return parser
    
    def predict_topk(self, dl, tokenizer, topk=4, bos_id=1, eos_id=2, max_out_len=26):
        utt2word= defaultdict(list)
        utt2logs = defaultdict(list)
        pbar = tqdm(dl)
        for batch in pbar:
            memory = self.backbone.forward_encoder(batch['feats'], 
                                              src_key_padding_mask=batch['src_key_padding_mask'])
            assert memory.shape[1] == 1, f"{memory.shape=}"
            # (SrcTime, Batch, E)
            tgt = torch.full(size=(1, 1), 
                             fill_value=bos_id, 
                             dtype=torch.long, 
                             device=memory.device)
            hyp_logprobs = torch.zeros((1), device=memory.device)
            tgt_ready = []
            mkpm = batch['src_key_padding_mask']
            for l in range(max_out_len):
                #print(f"{tgt.shape=}")
                tgt_logits = self.backbone.forward_decoder(tgt, 
                                                    memory.repeat(1, tgt.shape[1], 1), 
                                                    memory_key_padding_mask=mkpm.repeat((tgt.shape[1], 1)))
                tgt_logits = tgt_logits.log_softmax(dim=-1)
                
                new_tgt, logprobs = get_new_tgt(tgt, hyp_logprobs, tgt_logits[-1], topk=topk)
                ready, tgt, hyp_logprobs = sep_ready_tgt(new_tgt, logprobs)
                tgt_ready.extend(ready)
                if len(tgt_ready) >= topk:
                    break
                
            uid = batch['uids'][0]
            if len(tgt_ready) == 0:
                logging.warning(f"tgt_ready is 0 for {uid}. {tgt.shape=}. Use all hyps as ready hyps")
                tgt_ready = [(l.cpu().item(), t.cpu().tolist()) for l, t in zip(hyp_logprobs, tgt.T)]
                
            out_indices = []
            for logprob, indices in sorted(tgt_ready, reverse=True):
                joined = tokenizer.decode(indices) #.split()[0]
                utt2word[uid].append(joined)
                utt2logs[uid].append(logprob)
            d = '|'+'|'.join(utt2word[uid]) + "|"
            pbar.set_description(f"{d}\t".ljust(40, '='), refresh=False)
        return utt2word, utt2logs

In [151]:
sorted([1,2,3], reverse=True)

[3, 2, 1]

In [152]:
#                 nt_topk_logits, nt_topk_idx = tgt_logits.topk(k=topk, axis=-1)
#                 next_tokens = nt_topk_idx[-1:] 
#                 # (1, N, K)
#                 next_tokens = next_tokens.transpose(1, 2).reshape(1, -1)
#                 # (1, N*(repeat k times)) 
#                 new_tgt = tgt.repeat(1, topk)
#                 # (T, N*(repeat k times))  
#                 new_tgt = torch.concatenate([new_tgt, next_tokens], axis=0)
#                 #print(f"{new_tgt.shape}")
#                 # (T, N_new)
#                 new_scores = nt_topk_logits[-1].T.reshape(-1)
#                 prew_scores = tgt_logprobs.repeat(topk)
#                 tgt_logprobs = prew_scores + new_scores
                # (N,)
                
#                 is_end_mask = ((new_tgt == eos_id).sum(axis=0) > 0)
#                 # (N,)
#                 #print(is_end_mask)
#                 new_ready = new_tgt[:, is_end_mask]
#                 new_ready_logprobs = tgt_logprobs[is_end_mask]
#                 tgt_ready.extend([(l.cpu(), t.cpu()) for t, l in zip(new_ready.T, new_ready_logprobs)])
                
#                 #print(is_end)
#                 if len(tgt_ready) >= topk:
#                     break
                
#                 tgt = new_tgt[:, ~is_end_mask]
#                 tgt_logprobs = tgt_logprobs[~is_end_mask]

In [12]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/transformer_sc/lightning_logs/version_50424998/checkpoints/last.ckpt',backbone=model, map_location='cpu' )

PositionalEncoding shape is torch.Size([400, 1, 512])


In [165]:
utt2words, utt2logs = pl_module.predict_topk(val_dataloader, tokenizer=tokenizer, topk=10)

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-11-10 22:05:52,442 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


In [166]:
utt2words

defaultdict(list,
            {'valid-0': ['на',
              'нас',
              'нана',
              'наа',
              'на на',
              'нам',
              'наша',
              'напа',
              'га',
              'на-на',
              'на-а',
              'на нас'],
             'valid-1': ['все',
              'всем',
              'све',
              'вчера',
              'все-так',
              'вместе',
              'се',
              'сосе',
              'свет',
              'вме',
              'светлые',
              'все-ссе',
              'все-сак',
              'все-све',
              'все-с все',
              'все-с'],
             'valid-2': ['этом',
              'этом',
              'потом',
              'это',
              'дом',
              'отом',
              'этим',
              'этому',
              'этот',
              'жом',
              'жор'],
             'valid-3': ['добрый',
              'доброй',
              '

In [167]:
accuracy(valid_ref_u2w, {k:v[0] for k, v in utt2words.items()})
# topk2 total=10000 corr=8429 err=1571, accuracy: 0.8429
# topk5 total=10000 corr=8434 err=1566, accuracy: 0.8434
# topk10 total=10000 corr=8388 err=1612, accuracy: 0.8388

  0%|          | 0/10000 [00:00<?, ?it/s]

геев гены
была бача
рам нам
шакалов заказала
замазала запихала
воля волосы
ура уля
шорты шорту
баку бака
корень конечно
но но но но
говорить говорит
водитель водителю
вечером вечер
вот вот-то
фиолетовой фиолетовое
выехал выезжать
мы ивы
черна черная
выгуливать выгулить
сорян случая
пололи положи
вызовов вызово
пробовал попробовать
же желе
стать стараюсь
отвечал отвечать
ха за
русскому русским
мазок захотела
зачем звони
не на
обувь обед
завтра заработать
он лон
никогда никого
не неа
был было
баба батура
мойкой мойной
но но но
пойми пофиг
не нее
кн еген
виде видео
пахлава пахова
уезжай езжай
прошу проще
дура духов
агапкина агркина
дьявол дтаяк
ниже нижнее
верон веронику
анадырь аналогично
лада ладно
заберем заберет
завтра закрою
тыс там
тыкву тьфу
стоит строгит
ощущение результат
выбил фибир
не нее
пик пику
прошел пошел
романович романтиза
ирбит ирбить
глав глава
работы работаю
далеко жалко
доллар дождусь
занимаешься зарегистрировать
жень день
приветик привет
мм ммм
выглядят выглядит
кро

0.8388

In [21]:
with open('./data/voc.txt') as f:
    vocab = frozenset(s for s in map(str.strip, f.readlines()))

In [168]:
lv = {}
for k, v in utt2words.items():
    corr_w = None
    for w in v:
        if w in vocab:
            corr_w = w
            break
    if corr_w is None: 
        logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
        corr_w = '-'
    lv[k] = corr_w
accuracy(valid_ref_u2w, lv)
# topk10 total=10000 corr=8542 err=1458, accuracy: 0.8542


2023-11-10 22:27:02,886 root /tmp/ipykernel_2611601/1526573807.py:9 - WARNING - k='valid-321' doesn't have any vocab hyp. v=['дтаяк', 'дистая', 'дтрафа', 'дитаяр', 'дтаявки', 'дтрафет', 'дипломая', 'дтаявка', 'дтаяволь', 'дттаян', 'дтрафети']
2023-11-10 22:27:02,888 root /tmp/ipykernel_2611601/1526573807.py:9 - WARNING - k='valid-996' doesn't have any vocab hyp. v=['малохой', 'малоховый', 'малхоры', 'малхозы', 'малхозный', 'малохов', 'малохово', 'малхов', 'малохова', 'малоховой']
2023-11-10 22:27:02,892 root /tmp/ipykernel_2611601/1526573807.py:9 - WARNING - k='valid-3640' doesn't have any vocab hyp. v=['ребтишки', 'ребстки', 'ребишки', 'реббишки', 'ребьки', 'ребстришки', 'ребюстишки', 'ребстшки', 'ребстишки', 'ребстрашки', 'ребстришка', 'ребстрашение']
2023-11-10 22:27:02,895 root /tmp/ipykernel_2611601/1526573807.py:9 - WARNING - k='valid-6346' doesn't have any vocab hyp. v=['черковская', 'черкомская', 'черкосская', 'черековская', 'черкормская', 'черезская', 'черковка', 'черкомска', 

  0%|          | 0/10000 [00:00<?, ?it/s]

геев гены
была баса
рам нам
шакалов заказала
замазала запихала
воля волосы
ура уля
баку бака
корень конечно
говорить говорит
водитель водителю
вечером вечер
фиолетовой фиолетовое
выехал выезжать
мы ивы
черна черная
выгуливать выгулять
сорян случая
пололи положи
пробовал попробовать
же желе
стать стараюсь
отвечал отвечать
ха за
русскому русским
мазок захотела
зачем звони
не на
обувь обед
завтра заработать
он лон
никогда никого
не неа
был было
баба бабу
пойми пофиг
не нее
кн нечего
виде видео
уезжай езжай
прошу проще
дура духов
агапкина агрессии
дьявол 
ниже нижнее
верон веронику
анадырь аналогично
лада ладно
заберем заберет
завтра закрою
тыс там
тыкву тьфу
стоит строит
ощущение результат
выбил футболка
не нее
пик пику
прошел пошел
романович романтичная
глав глава
работы работаю
далеко жалко
доллар дождусь
занимаешься зарегистрировать
жень день
приветик привет
мм ммм
выглядят выглядит
крот корот
потому плитки
привете привет
позагорать познакомиться
ест есть
лежит делить
пойдут пойдет
дек

0.8542

In [169]:
test_ds =  FeatsIterableDatasetV2([f"ark:data_feats/test/feats.ark"], shuffle=False, 
                                 bos_id=1, 
                                 eos_id=2, 
                                 batch_first=False)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=1, collate_fn=test_ds.collate_pad)
#test_u2w = predict(pl_module.backbone, test_dataloader)
test_u2w, test_u2l = pl_module.predict_topk(test_dataloader, tokenizer=tokenizer, topk=8)

0it [00:00, ?it/s]

2023-11-10 23:31:47,024 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/test/feats.ark


In [170]:

def limit_vocab(u2w, vocab=vocab):
    lv = {}
    for k, v in u2w.items():
        corr_w = []
        for w in v:
            if w in vocab:
                corr_w.append(w)
        if len(corr_w) == 0: 
            logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
            corr_w = ['-']
        lv[k] = corr_w
    return lv
test_lv = limit_vocab(test_u2w)

2023-11-10 23:50:57,260 root /tmp/ipykernel_2611601/2474340753.py:9 - WARNING - k='test-187' doesn't have any vocab hyp. v=['оненененен', 'легенен', 'оненененене', 'оноененененен', 'легененененен', 'легенененен', 'оненененененен', 'огенененененен']
2023-11-10 23:50:57,266 root /tmp/ipykernel_2611601/2474340753.py:9 - WARNING - k='test-1319' doesn't have any vocab hyp. v=['госпориемка', 'госмотриемка', 'госпитаемка', 'гостерика', 'госпортика', 'госпортием', 'госпитариемка', 'гос-приемка', 'госпортиемка']
2023-11-10 23:50:57,269 root /tmp/ipykernel_2611601/2474340753.py:9 - WARNING - k='test-1670' doesn't have any vocab hyp. v=['алинатор', 'админатор', 'адмиратор', 'адмигатор', 'алишатор', 'алинатол', 'адигатор', 'алинато', 'алинаторна']
2023-11-10 23:50:57,269 root /tmp/ipykernel_2611601/2474340753.py:9 - WARNING - k='test-1700' doesn't have any vocab hyp. v=['каньков', 'канькова', 'канбель', 'канюков', 'кантура', 'каньков', 'канльков', 'канькой']
2023-11-10 23:50:57,270 root /tmp/ipyke

In [135]:
v, k = torch.topk(x, 2, axis=-1)
print(k)

k[1:, :, :].transpose(1,2).reshape(1, 2*3)

tensor([[[1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0]]])


tensor([[1, 1, 1, 0, 0, 0]])

In [137]:
torch.concatenate([x, k], axis=0)

tensor([[[1, 4],
         [1, 4],
         [2, 5]],

        [[2, 5],
         [3, 6],
         [3, 6]],

        [[1, 0],
         [1, 0],
         [1, 0]],

        [[1, 0],
         [1, 0],
         [1, 0]]])

In [8]:
!head ./exp/bpe500/model.vocab

<blk>	0
<s>	0
</s>	0
<unk>	0
▁п	-0
▁н	-1
▁с	-2
▁к	-3
▁т	-4
▁д	-5


In [13]:
val_ds = FeatsIterableDatasetV2([f"ark:data_feats/valid/feats.ark"], 
                             targets_rspecifier='ark:exp/bpe500/valid-text.int', 
                                shuffle=False,
                               bos_id=1, 
                               eos_id=2,
                               batch_first=False)
val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=1, collate_fn=val_ds.collate_pad)


2023-11-10 17:15:32,476 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:44 - INFO - Loading targets from ark:exp/bpe500/valid-text.int


Loading targets...: 0it [00:00, ?it/s]

In [76]:
with open('data_feats/valid/text') as f:
    valid_ref_u2w = {u:w for u, w in   map(str.split, f.readlines())}

In [9]:
%%time
for b in tqdm(val_dataloader):
    pass
print("Done")
print(b)

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-11-09 21:13:50,488 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:81: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  'feats': torch.as_tensor(feats, dtype=torch.float32),


Done
{'uids': ['valid-9999'], 'feats': tensor([[[1.0000, 0.3205, 0.2764,  ..., 1.0000, 0.4375, 0.7084]],

        [[1.0000, 0.3204, 0.2764,  ..., 1.0000, 0.4376, 0.7084]],

        [[1.0000, 0.3183, 0.2746,  ..., 1.0000, 0.4389, 0.7108]],

        ...,

        [[1.0000, 0.3608, 0.2706,  ..., 1.0000, 0.6705, 0.7665]],

        [[1.0000, 0.3603, 0.2701,  ..., 1.0000, 0.6704, 0.7667]],

        [[1.0000, 0.3603, 0.2701,  ..., 1.0000, 0.6704, 0.7667]]]), 'src_key_padding_mask': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [10]:
print(b['feats'].shape, b['targets'].shape)

torch.Size([174, 1, 37]) torch.Size([8, 1])


In [11]:
train_ds = FeatsIterableDatasetV2([f"ark:{f}" for f in sorted(glob("data_feats/train/feats.*.ark"))],
                                  targets_rspecifier='ark:exp/bpe500/train-text.int.ark', 
                                  shuffle=True,
                                  bos_id=1, 
                                  eos_id=2, 
                                 batch_first=False)

#train_ds = val_ds

#
# 35799.91it/s - txt format
# vs
# 136753.6it/s - ark format

2023-11-09 21:13:52,460 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:44 - INFO - Loading targets from ark:exp/bpe500/train-text.int.ark


Loading targets...: 0it [00:00, ?it/s]

In [12]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=24, collate_fn=train_ds.collate_pad, 
                                                num_workers=8)


In [14]:
trainer = pl.Trainer(max_epochs=6, log_every_n_steps=400, reload_dataloaders_every_n_epochs=1,
                    default_root_dir='exp/models/transformer_sc',
                    callbacks=[pl.callbacks.TQDMProgressBar(refresh_rate=100),
                              pl.callbacks.ModelCheckpoint(every_n_train_steps=20000,
                                                          save_last=True)],
                    accumulate_grad_batches=4, 
                    check_val_every_n_epoch=1)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer(backbone=model)

PositionalEncoding shape is torch.Size([400, 1, 512])


In [18]:
with torch.no_grad():
    loss = pl_module.get_loss(b)
print(loss)

tensor(0.6119)


In [116]:
for b in train_dataloader:
    pass

2023-11-08 23:34:29,733 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


In [125]:
with torch.no_grad():
    pl_module.eval()
    feats = b['feats']
    # (Batch, Time, num_feats)
    tgt = b['targets'][:, :-1]
    print(feats.shape, b['targets'].shape, tgt.shape)
    tgt_key_padding_mask = b['tgt_key_padding_mask'][:, :-1] 
    #tgt_key_padding_mask = b['tgt_key_padding_mask'][:, :-1]
    # (Batch, Seq-1)
    logits = pl_module.backbone(feats=feats, 
                           tgt=tgt, 
                           src_key_padding_mask=b['src_key_padding_mask'], 
                           tgt_key_padding_mask=tgt_key_padding_mask) 
    # (Seq-1, Batch, C)
    S, N, C = logits.shape
    targets = b['targets'][:, 1:].T
    #tgt_key_padding_mask 
    # (Seq-1, Batch)
    print("loss ", logits.shape, targets.shape)
    loss = pl_module.ce_loss(logits.view(-1, C), targets.reshape(-1))
print(loss)

torch.Size([16, 174, 37]) torch.Size([16, 8]) torch.Size([16, 7])
loss  torch.Size([7, 16, 500]) torch.Size([7, 16])
tensor(3.4366)


In [127]:
with torch.no_grad():
    pl_module.eval()
    l = []
    t = []
    for i in range(b['feats'].shape[0]):
        feats = b['feats'][i:i+1]
        # (Batch, Time, num_feats)
        tgt = b['targets'][i:i+1, :-1]
        print(feats.shape, b['targets'].shape, tgt.shape)
        tgt_key_padding_mask = b['tgt_key_padding_mask'][i:i+1, :-1] 
        #tgt_key_padding_mask = b['tgt_key_padding_mask'][:, :-1]
        # (Batch, Seq-1)
        logits = pl_module.backbone(feats=feats, 
                               tgt=tgt, 
                               src_key_padding_mask=b['src_key_padding_mask'][i:i+1], 
                               tgt_key_padding_mask=tgt_key_padding_mask) 
        # (Seq-1, Batch, C)
        S, N, C = logits.shape
        targets = b['targets'][i:i+1, 1:].T
        #tgt_key_padding_mask 
        # (Seq-1, Batch)
        print("loss ", logits.shape, targets.shape)
        l.append(logits.view(-1, C))
        t.append(targets.reshape(-1))
    loss = pl_module.ce_loss(torch.concatenate(l), torch.concatenate(t))
print(loss)

torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) torch.Size([7, 1])
torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) torch.Size([7, 1])
torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) torch.Size([7, 1])
torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) torch.Size([7, 1])
torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) torch.Size([7, 1])
torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) torch.Size([7, 1])
torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) torch.Size([7, 1])
torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) torch.Size([7, 1])
torch.Size([1, 174, 37]) torch.Size([16, 8]) torch.Size([1, 7])
loss  torch.Size([7, 1, 500]) to

In [124]:
lt, tt = torch.concatenate(l), torch.concatenate(t)
print(lt.shape, tt.shape)
lt = lt[tt!=0]
tt = tt[tt!=0]
print(lt.shape, tt.shape)
torch.nn.CrossEntropyLoss()(lt, tt)

torch.Size([112, 500]) torch.Size([112])
torch.Size([51, 500]) torch.Size([51])


tensor(3.4949)

In [97]:
#trainer.fit(pl_module, val_dataloader, val_dataloader)

In [13]:
trainer.fit(pl_module, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                  | Params
---------------------------------------------------
0 | backbone | TransformerWithSinPos | 44.7 M
1 | ce_loss  | CrossEntropyLoss      | 0     
---------------------------------------------------
44.7 M    Trainable params
0         Non-trainable params
44.7 M    Total params
178.690   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

2023-11-09 00:57:18,412 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:120: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(
/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:81: UserWarning: The given NumPy array is not writa

2023-11-09 00:57:20,014 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.108.ark
2023-11-09 00:57:20,013 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.93.ark
2023-11-09 00:57:20,029 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.110.ark
2023-11-09 00:57:20,015 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.80.ark
2023-11-09 00:57:20,013 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.63.ark
2023-11-09 00:57:20,016 root /mnt

...

2023-11-09 17:28:08,699 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.4.ark
2023-11-09 17:28:08,820 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.55.ark
2023-11-09 17:52:21,450 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.27.ark
2023-11-09 17:52:21,550 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.93.ark
2023-11-09 17:52:21,648 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.58.ark
2023-11-09 17:52:21,764 root /mnt/as

2023-11-09 19:53:49,588 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.86.ark
2023-11-09 19:53:49,684 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.43.ark
2023-11-09 19:53:49,821 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.22.ark
2023-11-09 19:53:49,915 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.81.ark
2023-11-09 19:53:50,021 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.31.ark
2023-11-09 19:53:50,112 root /mnt/a

Validation: 0it [00:00, ?it/s]

2023-11-09 20:18:08,577 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark
2023-11-09 20:19:55,154 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.53.ark
2023-11-09 20:19:55,151 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.22.ark
2023-11-09 20:19:55,149 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.20.ark
2023-11-09 20:19:55,155 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.36.ark
2023-11-09 20:19:55,152 root /mnt/asr_

/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [16]:
trainer

In [19]:
#result = trainer.test(pl_module, train_dataloader)
print(result)

NameError: name 'trainer' is not defined

In [21]:
result = trainer.test(pl_module, val_dataloader)
print(result)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:120: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

2023-11-09 21:18:35,620 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.20462335646152496    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.20462335646152496}]


In [1]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/transformer_sc/lightning_logs/version_50424998/checkpoints/last.ckpt',backbone=model, map_location='cpu' )

NameError: name 'TransformerWithSinPos' is not defined

In [17]:
trainer = pl.Trainer(max_epochs=2, log_every_n_steps=400, reload_dataloaders_every_n_epochs=1,
                    default_root_dir='exp/models/transformer_sc',
                    callbacks=[pl.callbacks.TQDMProgressBar(refresh_rate=100),
                              pl.callbacks.ModelCheckpoint(every_n_train_steps=10000,
                                                          save_last=True)],
                    accumulate_grad_batches=4, 
                    check_val_every_n_epoch=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [83]:
result = trainer.test(pl_module, val_dataloader)
print(result)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:120: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

2023-11-09 23:19:24,689 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.20462335646152496    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.20462335646152496}]


In [84]:
trainer.fit(pl_module, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                  | Params
---------------------------------------------------
0 | backbone | TransformerWithSinPos | 44.7 M
1 | ce_loss  | CrossEntropyLoss      | 0     
---------------------------------------------------
44.7 M    Trainable params
0         Non-trainable params
44.7 M    Total params
178.690   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

2023-11-09 23:20:50,825 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


2023-11-09 23:20:51,826 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.113.ark
2023-11-09 23:20:51,826 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.71.ark
2023-11-09 23:20:51,826 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.22.ark
2023-11-09 23:20:51,826 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.108.ark
2023-11-09 23:20:51,882 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.16.ark
2023-11-09 23:20:51,886 root /mnt

...

2023-11-10 09:12:58,783 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.76.ark
2023-11-10 09:12:58,904 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.19.ark
2023-11-10 09:36:34,389 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.34.ark
2023-11-10 09:36:34,458 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.42.ark
2023-11-10 09:36:34,562 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.72.ark
2023-11-10 09:36:34,652 root /mnt/a

2023-11-10 11:34:40,076 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.86.ark
2023-11-10 11:34:40,142 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.87.ark
2023-11-10 11:34:40,212 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.22.ark
2023-11-10 11:34:40,305 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.89.ark
2023-11-10 11:34:40,383 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/train/feats.97.ark
2023-11-10 11:34:40,462 root /mnt/a

Validation: 0it [00:00, ?it/s]

2023-11-10 11:58:10,997 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


`Trainer.fit` stopped: `max_epochs=2` reached.


In [85]:
result = trainer.test(pl_module, val_dataloader)
print(result)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing: 0it [00:00, ?it/s]

2023-11-10 12:01:18,185 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.20782829821109772    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.20782829821109772}]


In [9]:
L = torch.LongTensor([3, 4, 2, 6, 5, 1, 3, 2, 4, 7])
M = torch.arange(20) < L[:, None]
M

tensor([[ True,  True,  True, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True, False, False, Fa

In [10]:
M.shape

torch.Size([10, 20])

In [44]:
import sentencepiece as spm
import math

In [45]:
tokenizer = spm.SentencePieceProcessor('exp/bpe500/model.model')

In [46]:
tokenizer.decode([[10, 11, 12], [12, 13, 15]])

['в о по', 'по мак']

In [65]:
def predict(backbone, dl, bos_id=1, eos_id=2, max_out_len=10):

    utt2word={}
    pbar = tqdm(dl)
    for batch in pbar:
        src_embs = backbone.input_ff(batch['feats'])
        # (S, B, E)
        src_embs = backbone.positional_encoding(src_embs)
        memory = backbone.transformer.encoder(src_embs, src_key_padding_mask=batch['src_key_padding_mask'])
        # (Time, Batch, E)
        tgt = torch.full(size=(1, memory.shape[1]), fill_value=bos_id, dtype=torch.long, device=memory.device)
        for _ in range(max_out_len):
            tgt_embs = backbone.tgt_embedding(tgt) * math.sqrt(backbone.d_model)
            tgt_embs = backbone.positional_encoding(tgt_embs)
            mask = pl_module.backbone.transformer.generate_square_subsequent_mask(tgt.shape[0], device=tgt.device)
            #print(tgt_embs.shape, memory.shape)
            next_tokens = backbone.transformer.decoder(tgt_embs, memory, tgt_mask=mask)
            next_tokens = backbone.head(next_tokens).argmax(dim=-1)

            #print(tgt.shape, next_tokens.shape)
            tgt = torch.concatenate([tgt, next_tokens[-1:]], axis=0)
            #print(tgt)
            is_end = ((tgt == eos_id).sum(axis=0) > 0).all()
            #print(is_end)
            if is_end:
                break

        for uid, indices in zip(batch['uids'], tgt.T):
            out_indices = []
            for i in indices.tolist():
                out_indices.append(i)
                if i == eos_id:
                    break

            # torch.unique_consecutive(indices, dim=-1).tolist()
            #print(indices)
            joined = tokenizer.decode(out_indices).split()[0]
            pbar.set_description(f"{joined}", refresh=False)
            utt2word[uid] = joined
    return utt2word

In [101]:
def predict_v2(backbone: TransformerWithSinPos, dl, bos_id=1, eos_id=2, max_out_len=10):

    utt2word={}
    pbar = tqdm(dl)
    for batch in pbar:
        memory = backbone.forward_encoder(batch['feats'], 
                                          src_key_padding_mask=batch['src_key_padding_mask'])
        # (SrcTime, Batch, E)
        tgt = torch.full(size=(1, memory.shape[1]), 
                         fill_value=bos_id, 
                         dtype=torch.long, 
                         device=memory.device)
        for _ in range(max_out_len):
            tgt_embs = backbone.forward_decoder(tgt, 
                                                memory, 
                                                memory_key_padding_mask=batch['src_key_padding_mask'])

            next_tokens = tgt_embs.argmax(dim=-1)

            #print(tgt.shape, next_tokens.shape)
            tgt = torch.concatenate([tgt, next_tokens[-1:]], axis=0)
            #print(tgt)
            is_end = ((tgt == eos_id).sum(axis=0) > 0).all()
            #print(is_end)
            if is_end:
                break

        for uid, indices in zip(batch['uids'], tgt.T):
            out_indices = []
            for i in indices.tolist():
                out_indices.append(i)
                if i == eos_id:
                    break

            # torch.unique_consecutive(indices, dim=-1).tolist()
            #print(indices)
            joined = tokenizer.decode(out_indices).split()[0]
            pbar.set_description(f"{joined}", refresh=False)
            utt2word[uid] = joined
    return utt2word

In [17]:
#valid_u2w = predict_v2(pl_module.backbone, val_dataloader)
with open('data_feats/valid/text') as f:
    valid_ref_u2w = {u:w for u, w in   map(str.split, f.readlines())}
    

In [16]:
def accuracy(ref_u2w, hyp_u2w):
    corr = 0
    err = 0
    total = len(ref_u2w)
    for u, ref in tqdm(ref_u2w.items()):
        hyp = hyp_u2w[u].strip('-')
        if ref != hyp:
            print(ref, hyp)
            err +=1
        else:
            corr +=1
    a = corr/total
    print(f"{total=} {corr=} {err=}, accuracy: {a}")
    return a
#accuracy(valid_ref_u2w, valid_u2w)
# total=10000 corr=8227 err=1773, accuracy: 0.8227


In [88]:
test_ds =  FeatsIterableDatasetV2([f"ark:data_feats/test/feats.ark"], shuffle=False, 
                                 bos_id=1, 
                                  eos_id=2, 
                                 batch_first=False)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=1, collate_fn=test_ds.collate_pad)
test_u2w = predict(pl_module.backbone, test_dataloader)

0it [00:00, ?it/s]

2023-11-10 14:08:53,132 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/test/feats.ark


In [89]:
test_u2w

{'test-0': 'на',
 'test-1': 'что',
 'test-2': 'опоздания',
 'test-3': 'сколько',
 'test-4': 'дремать',
 'test-5': 'не',
 'test-6': 'как',
 'test-7': 'садовод',
 'test-8': 'заметил',
 'test-9': 'ваги',
 'test-10': 'ок',
 'test-11': 'плинтус',
 'test-12': 'ай',
 'test-13': 'ищем',
 'test-14': 'лет',
 'test-15': 'могу',
 'test-16': 'может',
 'test-17': 'спокойной',
 'test-18': 'рядов',
 'test-19': 'вспомнить',
 'test-20': 'максим',
 'test-21': 'веселое',
 'test-22': 'невиномысла',
 'test-23': 'туда',
 'test-24': 'тебя',
 'test-25': 'ре',
 'test-26': 'точно',
 'test-27': 'чего',
 'test-28': 'помою',
 'test-29': 'хорошо',
 'test-30': 'укладки',
 'test-31': 'нужны',
 'test-32': 'ты',
 'test-33': 'почтушенным',
 'test-34': 'не',
 'test-35': 'поеду',
 'test-36': 'то',
 'test-37': 'быть',
 'test-38': 'не',
 'test-39': 'завтраки',
 'test-40': 'будем',
 'test-41': 'дома',
 'test-42': 'со',
 'test-43': 'свою',
 'test-44': 'оне',
 'test-45': 'было',
 'test-46': 'человек',
 'test-47': 'погоди',
 'te

In [172]:
baseline_result = pd.read_csv('exp/models/ctc_trans/lightning_logs/version_50422251/test_submit.v1.csv', sep=',', names=['main', 'second', 'third', 'trash'])
baseline_result['uid'] = [f'test-{i}' for i in range(len(baseline_result))]
baseline_result.head()

,main,second,third,trash,uid
0,на,неа,на,ненка,test-0
1,что,часто,частого,чисто,test-1
2,опоздания,опозданиям,оприходования,опозданиями,test-2
3,сколько,сокольского,свердловского,скроено,test-3
4,дремать,дописать,донимать,дюрренматт,test-4


In [92]:
baseline_result['predict'] = baseline_result.uid.apply(lambda x: test_u2w[x].strip('-'))
baseline_result.head()

,main,second,third,trash,uid,predict
0,на,неа,на,ненка,test-0,на
1,что,часто,частого,чисто,test-1,что
2,опоздания,опозданиям,оприходования,опозданиями,test-2,опоздания
3,сколько,сокольского,свердловского,скроено,test-3,сколько
4,дремать,дописать,донимать,дюрренматт,test-4,дремать


In [93]:
baseline_result[baseline_result['main']!= baseline_result['predict']]

,main,second,third,trash,uid,predict
11,плинциса,плинтус,потренируемся,поинтересуемся,test-11,плинтус
14,летет,орет,огреет,огреть,test-14,лет
18,рядоды,распродав,распроданы,распродавая,test-18,рядов
19,вспоминать,вспоминает,вспомнилась,вспоминалась,test-19,вспомнить
22,невиномыск,невинномысск,невинномысске,невинномысска,test-22,невиномысла
...,...,...,...,...,...,...
9985,не,нее,не,нк,test-9985,нее
9989,хахаха,хлора,хлопа,хлорка,test-9989,ха
9993,корччи,кололись,колорист,кромсать,test-9993,кормят
9994,буду,бруцелл,бруно,борецкого,test-9994,булл


In [94]:
rows = []

for i, row in baseline_result.iterrows():
    old_main = row['main']
    new_main = row['predict']
    if new_main != old_main:
        new_s = old_main
        new_th = row['second']
        new_tr = row['third']
    else:
        new_s = row['second']
        new_th = row['third']
        new_tr = row['trash']
    rows.append({"main": new_main,
                "second": new_s,
                "third": new_th,
                "trash": new_tr})
        
submission = pd.DataFrame(rows)
submission.head()


,main,second,third,trash
0,на,неа,на,ненка
1,что,часто,частого,чисто
2,опоздания,опозданиям,оприходования,опозданиями
3,сколько,сокольского,свердловского,скроено
4,дремать,дописать,донимать,дюрренматт


In [95]:
submission.to_csv("exp/models/transformer_sc/lightning_logs//version_50432204/test_submit.v3.csv", 
                  sep=',', header=False, index=False)

In [173]:
baseline_result['predict'] = baseline_result.uid.apply(lambda u: test_lv[u])
baseline_result.head()

,main,second,third,trash,uid,predict
0,на,неа,на,ненка,test-0,"[на, нас, нана, нага]"
1,что,часто,частого,чисто,test-1,"[что, что-то, сто, чисто, чтоб, часто, со, чмок]"
2,опоздания,опозданиям,оприходования,опозданиями,test-2,"[опоздания, опозданиям, опоздание, опоздании, ..."
3,сколько,сокольского,свердловского,скроено,test-3,"[сколько, скольки, сколько, сколько, столько, ..."
4,дремать,дописать,донимать,дюрренматт,test-4,"[дремать, донимать, думать, ждем-с]"


In [177]:
rows = []

for i, row in baseline_result.iterrows():
    ps = row['predict']
    for p in [row['main'], row['second'], row['third'], row['trash']]:
        if p not in ps:
            ps.append(p)
    rows.append(ps[:4])
        
submission = pd.DataFrame(rows, columns=['main', 'second', 'third', 'trash'])
submission.head()

,main,second,third,trash
0,на,нас,нана,нага
1,что,что-то,сто,чисто
2,опоздания,опозданиям,опоздание,опоздании
3,сколько,скольки,сколько,сколько
4,дремать,донимать,думать,ждем-с


In [178]:
submission.to_csv("exp/models/transformer_sc/lightning_logs/version_50424998/test_submit.v4.csv", 
                  sep=',', header=False, index=False)

In [15]:
s5 = pd.read_csv('exp/models/transformer_sc/lightning_logs/version_50424998/test_submit.v5.csv', sep=',', names=['main', 'second', 'third', 'trash'])
s5['uid'] = [f'test-{i}' for i in range(len(s5))]
s5.head()

,main,second,third,trash,uid
0,на,нас,нана,нам,test-0
1,что,что-то,сто,чтоб,test-1
2,опоздания,опоздание,опозданиям,опозданий,test-2
3,сколько,скольки,сколько,столько,test-3
4,дремать,донимать,думать,дописать,test-4


PositionalEncoding shape is torch.Size([400, 1, 512])


In [20]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/transformer_sc/lightning_logs/version_50424998/checkpoints/last.ckpt',backbone=model, map_location='cpu' )

val_ds = FeatsIterableDatasetV2([f"ark:data_feats/valid/feats.ark"], 
                                 targets_rspecifier='ark:exp/bpe500/valid-text.int', 
                                 shuffle=False,
                                 bos_id=1, 
                                 eos_id=2,
                                 batch_first=False)
val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=1, collate_fn=val_ds.collate_pad)
utt2words, utt2logs = pl_module.predict_topk(val_dataloader, tokenizer=tokenizer, topk=10)

2023-11-11 12:43:19,108 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:44 - INFO - Loading targets from ark:exp/bpe500/valid-text.int


Loading targets...: 0it [00:00, ?it/s]

In [ ]:
lv = {}
for k, v in utt2words.items():
    corr_w = None
    for w in v:
        if w in vocab:
            corr_w = w
            break
    if corr_w is None: 
        logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
        corr_w = '-'
    lv[k] = corr_w
accuracy(valid_ref_u2w, lv)
# topk10 total=10000 corr=8542 err=1458, accuracy: 0.8542